In [25]:
%%writefile query_api.py
from fastapi import FastAPI, Query, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import sqlite3
from pathlib import Path

# === 你的 SQLite 資料庫檔名 ===
DB_PATH = Path("osh_incidents.db")

# === 建立 FastAPI App ===
app = FastAPI(title="OSH Query API")


# ---------- Pydantic Response Models ----------

class Incident(BaseModel):
    uid: int
    industry: Optional[str]
    incident: Optional[str]
    description_summary: Optional[str]
    cause_summary: Optional[str]


class SearchResult(BaseModel):
    keyword: str
    count: int
    results: List[Incident]


# ---------- 資料庫連線 ----------

def connect_db():
    if not DB_PATH.exists():
        raise HTTPException(status_code=500, detail=f"找不到資料庫：{DB_PATH}")
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn


# ---------- API 定義 ----------

@app.get("/health")
def health_check():
    """用來測 API 是否還活著"""
    return {"status": "ok", "db_exists": DB_PATH.exists()}


@app.get("/search", response_model=SearchResult)
def search(keyword: str = Query(...), limit: int = Query(5, ge=1, le=50)):
    """
    用 keyword 搜尋 description_summary, cause_summary, incident
    """
    conn = connect_db()

    like = f"%{keyword}%"
    sql = """
        SELECT uid, industry, incident,
               description_summary, cause_summary
        FROM incidents
        WHERE description_summary LIKE ?
           OR cause_summary LIKE ?
           OR incident LIKE ?
        ORDER BY uid DESC
        LIMIT ?;
    """

    cur = conn.execute(sql, (like, like, like, limit))
    rows = cur.fetchall()
    conn.close()

    results = [
        Incident(
            uid=row["uid"],
            industry=row["industry"],
            incident=row["incident"],
            description_summary=row["description_summary"],
            cause_summary=row["cause_summary"]
        )
        for row in rows
    ]

    return SearchResult(keyword=keyword, count=len(results), results=results)

Overwriting query_api.py


In [27]:
!nohup uvicorn query_api:app --host 0.0.0.0 --port 8000 > server.log 2>&1 &


In [28]:
import requests
requests.get("http://127.0.0.1:8000/health").json()


{'status': 'ok'}

In [29]:
from pyngrok import ngrok

public_url = ngrok.connect(8000)
public_url


<NgrokTunnel: "https://gustoish-cuc-unvariegated.ngrok-free.dev" -> "http://localhost:8000">